In [ ]:
# dataset
import pandas as pd
import numpy as np
pd.set_option('display.max_columns' , None)
pd.set_option('display.max_rows' , None)
pd.set_option('display.max_colwidth', None)

pd.options.mode.copy_on_write = True

print(pd.options.mode.copy_on_write)

url = 'https://en.wikipedia.org/wiki/Historical_rankings_of_presidents_of_the_United_States'
raw = pd.read_html(url)
data = raw[3]
data
data = data.to_csv(r'C:\Users\fengz\Downloads\data.csv')
df = pd.read_csv(r'C:\Users\fengz\Downloads\data.csv', index_col=0)

# update value by locating the index and column
df.iloc[21, 0] = 22 
# add a new row by copy from an existing row
df.loc[len(df)] = df.iloc[21]
# update value by locating the index and column
df.iloc[-1:, 0] = 24



In [ ]:
import numpy as np
import pandas as pd


def tweak_data(data):
	return (data
			.rename(columns= {'Seq.': 'seq'}) 
			.rename(columns= {k: v.replace(' ', '_') for k, v in  
				{'Political party': 'party',
				'Bg': 'Background',
				'Im': 'Imagination',
				'Int': 'Integrity',
				'IQ': 'Intelligence',
				'L': 'Luck',
				'WR': 'Willing to take risks',
				'AC': 'Ability to compromise',
				'EAb': 'Executive ability',
				'LA': 'Leadership ability',
				'CAb': 'Communication ability',
				'OA': 'Overall ability',
				'PL': 'Party leadership',
				'RC': 'Relations with Congress',
				'CAp': 'Court appointments',
				'HE': 'Handling of economy',
				'EAp': 'Executive appointments',
				'DA': 'Domestic accomplishments',
				'FPA': 'Foreign policy accomplishments',
				'AM': 'Avoid crucial mistakes',
				'EV': "Experts’ view",
				'O': 'Overall'}.items()
				})
			# passing function .lower itself to rename(cplumns=...)
			# tells python to apply str.lower to each col name	
			# Use str.lower (without parentheses) when passing it as an argument to functions 
			# like map(), rename(), or apply().								
			.pipe(lambda df_: df_.rename(columns= str.lower))
			.assign(party = lambda df_: df_.party	
											.str.replace('-', ' ')
											.str.replace(r'\[.*\]', '', regex=True)
											.str.lower()
											.astype('category'),
					seq = lambda df_: df_.seq.astype('int'),
					average_rank = lambda df_: df_
											.select_dtypes('int64')
											.sum(axis='columns')   
											.rank(method='dense') 
											.astype('int'),  
					quartile= lambda df_: pd.qcut(df_.average_rank, 4, 
											 labels='1st,2nd,3rd,4th'.split(','))
												# labels=['1st', '2nd', '3rd', '4th'])
										
										)
			.sort_values('seq')
		#	.pipe(lambda df_: df_.astype({col: 'uint8[pyarrow]' for col in df_.select_dtypes('int64')}))
			# same result
			#.pipe(lambda df_: df_.astype({col: 'uint8[pyarrow]' for col in df_.select_dtypes('int64').columns}))
		)


# load the tweak func
#tweak_data(df)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


fig, ax = plt.subplots(figsize=(10, 10), dpi=600)

g = sns.heatmap(tweak_data(df)
   .set_index('president')
   .loc[:, 'background': 'average_rank'],
        annot= True, ax = ax, cmap = 'cividis')

g.set_xticklabels(g.get_xticklabels(), rotation = 45, fontsize=8, ha='right')

_=plt.title('President Ranking')

# plt.show()

In [ ]:
pres = tweak_data(df)

pres.info()
pres.columns

(pres
	.select_dtypes('number')
	.agg(['count', 'size', 'sum', lambda col: col.loc[1]])

)

(pres
 	.agg({'luck':  ['count', 'size', 'max'],
         'overall': ['count', 'size', 'max']
	   
	   }))


(pres
 	.agg(intelligence_count=('intelligence', 'count'),
        intelligence_size=('intelligence', 'size')
	   )
    .reset_index()
    .rename(columns={'index': 'indicator'})
)


(pres
    .sort_values(by='president', key=lambda name_ser: name_ser
									 .str.split(' ').str[-1]
                              ,ignore_index=True)       
                                     
    .sort_index(axis=1)
    )



(pres
	.set_index('president')
    .rename(index=lambda x: f'{x.split(' ')[0][0]}. {' '.join(x.split(' ')[1:])}')
    .index
            
	
)


(pres
	.loc[(pres.party == 'republican')
          & (pres['average_rank'] < 3),
       #   ['seq', 'president', 'party']
		lambda df_: df_.columns[:3]
		  
		  ]
 

)


# (pres
# 	.query('(party == "republican") and (average_rank < 3)')
#     .loc[:, ['president','party','background','party_leadership']]
#    # .iloc[:, :5]
	
# )


In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns' , None)
pd.set_option('display.max_rows' , None)
pd.set_option('display.max_colwidth', None)


url = 'https://github.com/mattharrison/datasets/raw/master/data/2020-jetbrains-python-survey.csv'
survey = pd.read_csv(url)


# survey['company.size'].value_counts(dropna=False)
# survey['employment.status'].value_counts(dropna=False)
# survey['is.python.main'].value_counts(dropna=False)
# survey['team.size'].value_counts(dropna=False)
# survey['years.of.coding'].value_counts(dropna=False)
# survey['python.years'].value_counts(dropna=False)
# survey['python3.version.most'].value_counts(dropna=False)
# survey['use.python.most'].value_counts(dropna=False)

# survey['nps.main.ide'].value_counts(dropna=False)



In [ ]:
# survey = (survey
# 	.rename(columns = lambda col: col.replace('.', '_'))
# 	.assign(are_you_datascientist = lambda df_: df_
# 								.are_you_datascientist                      
# 								.replace({'Yes': '1', 'No': '0', '':'0', 'Other': '0'})
# 								.astype('bool'),
                                
# 			use_python_most = lambda df_: df_
# 								.use_python_most
# 								.fillna('Unknown'),
                                
# 	 		python3_version_most = lambda df_: df_  
# 	 							.python3_version_most
# 	 							.str.replace('_', '.') #substring replace
# 								.str.extract(r'(\d+\.\d+)')
#                                 .fillna('Unknown')    
# 								.astype('category'),
                                
# 			python_years = lambda df_:df_
# 								.python_years  
#                                 .replace('Less than 1 year', '<1') 
# 								.str.extract(r'(<\d+|\d+\–\d+|\d+\+)')
#                                 .fillna('Unknown')    
# 								.astype('category'),
						         
# 			years_of_coding = lambda df_:df_
# 								.years_of_coding
# 								.replace('Less than 1 year', '<1')   
# 								.str.extract(r'(<\d+|\d+\–\d+|\d+\+)')
#                                 .fillna('Unknown')    
# 								.astype('category'),
	 
# 			is_python_main = lambda df_:df_ 
# 								.is_python_main
# 								.fillna('Unknown')    
# 								.astype('category'),
                                
# 			employment_status = lambda df_:df_ 
# 								.employment_status
# 								.fillna('Unknown')   
# 								.astype('category'),
                                
# 			company_size = lambda df_: df_
# 								.company_size
# 								.replace({'Just me': '1',
# 										'Not sure': 'Unknown',
# 										'More than 5,000': '5000+'
# 										})	
#                                 .str.replace(',', '')	
# 								.astype('category'),
                                
# 			team_size = lambda df_: df_
# 								.team_size
# 								.replace('More than 40 people', '40+')
# 								.str.extract(r'(\d+\+|\d+\-\d+)')
#                                 .fillna('Unknown')   
# 								.astype('category')
# 	 	)                     


# )




In [ ]:
# def combine_cols(data, substrings):
#     return {substring: 
# 					(data
# 						.filter(axis=1, like=substring)
# 						.where(data.isna(), '1')
# 						.fillna('0')
# 						.astype('int')
# 						.astype('bool')
# 						.idxmax(axis=1)
# 						.str.replace(substring, '') # categorical-like Series. ready for dummy cols
					
# 						) for substring in substrings
# 			}			


# substrings = [
# 		'job_role_', 
#         'other_lang_', 
#         'other_purposes_', 
#         'how_involved_',
# 		'web_frameworks_', 
#         'data_frameworks_', 
#         'other_frameworks_', 
#         'database_',
# 		'bigdata_', 
#         'ide_editor_', 
#         'configuration_management_', 
#         'ci_',
#         'cloud_platform_',
#         'isolate_environments_',
# 		'unittesting_',
#         'orm_',
#         'develop_for_cloud_',
#         'run_in_cloud_',
#         'python_version_upgrade_',
#         'devenv_os_'
    
# ]

# combined_results = combine_cols(survey, substrings)




# (survey
# 		.loc[:, ['age', 'are_you_datascientist', 'company_size',
# 					'country_live', 'employment_status',
# 					'first_learn_about_main_ide', 'how_often_use_main_ide',
# 					'ide_main', 'is_python_main', 'job_team', 'main_purposes',
# 					'missing_features_main_ide', 'nps_main_ide', 'python_years',
# 					'python3_version_most', 'several_projects', 'team_size',
# 					'use_python_most', 'years_of_coding']]
                    
# # 	.loc[:, ~survey.columns.str.contains('job_role', regex=True)]

#     .assign(job_role=combined_results['job_role_'],
#             other_lang=combined_results['other_lang_'],
#             other_purposes = combined_results['other_purposes_'],
#             how_involved = combined_results['how_involved_'],
#             web_frameworks = combined_results['web_frameworks_'],
#             data_frameworks = combined_results['data_frameworks_'],
#             other_frameworks = combined_results['other_frameworks_'],
# 			database = combined_results['database_'],
# 			bigdata = combined_results['bigdata_'],
# 			ide_editor = combined_results['ide_editor_'], 
# 			configuration_management = combined_results['configuration_management_'], 
# 			ci = combined_results['ci_'],
# 			cloud_platform = combined_results['cloud_platform_'],
# 			isolate_environments = combined_results['isolate_environments_'],
# 			unittesting	= combined_results['unittesting_'],
# 			orm	= combined_results['orm_'],
# 			develop_for_cloud = combined_results['develop_for_cloud_'],
# 			run_in_cloud = combined_results['run_in_cloud_'],
# 			python_version_upgrade = combined_results['python_version_upgrade_'],
# 			devenv_os = combined_results['devenv_os_']

# 	)


			
	
			
# )






In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns' , None)
pd.set_option('display.max_rows' , None)
pd.set_option('display.max_colwidth', None)



def tweak_survey(data, substrings=None):
	'''Cleans and transform survey data'''
    
	# feature transformation
	data = (data
				.rename(columns = lambda col: col.replace('.', '_')) 
				.fillna('Unknown')
				.replace('–', '-')
				.assign(are_you_datascientist = lambda df_: df_
											.are_you_datascientist                      
											.replace({'Yes': '1', 'No': '0', '':'0', 'Other': '0'})
											.astype('bool'),
											
						# use_python_most = lambda df_: df_
						# 					.use_python_most,
						# 				#	.fillna('Unknown'),
											
						python3_version_most = lambda df_: df_  
											.python3_version_most
											.str.replace('_', '.') #substring replace
											.str.extract(r'(\d+\.\d+)'),
										#	.fillna('Unknown')    
											#.astype('category'),
											
						python_years = lambda df_:df_
											.python_years  
											.replace('Less than 1 year', '<1') 
											.str.extract(r'(<\d+|\d+\-\d+|\d+\+)'),
										#	.fillna('Unknown')    
											# .astype('category'),
											
						years_of_coding = lambda df_:df_
											.years_of_coding
											.replace('Less than 1 year', '<1')   
											.str.extract(r'(<\d+|\d+\-\d+|\d+\+)'),
									#		.fillna('Unknown')    
											# .astype('category'),
				
						# is_python_main = lambda df_:df_ 
						# 					.is_python_main
						# 			#		.fillna('Unknown')    
						# 					.astype('category'),
											
						# employment_status = lambda df_:df_ 
						# 					.employment_status
						# 			#		.fillna('Unknown')   
						# 					.astype('category'),
											
						company_size = lambda df_: df_
											.company_size
											.replace({'Just me': '1',
													'Not sure': 'Unknown',
													'More than 5,000': '5000+'
													})	
											.str.replace(',', ''),	
											# .astype('category'),
											
						team_size = lambda df_: df_
											.team_size
											.replace('More than 40 people', '40+')
											.str.extract(r'(\d+\+|\d+\-\d+)'),
									#		.fillna('Unknown')   
											# .astype('category'),

						# taking the avg of the age range
						age = lambda df_: ((df_
												.age
												.replace('Unknown', '0')
												.str.extract(r'(\d+|\d+\-)') # extract the first number
												.astype('int'))
											.add((df_
												.age
												.replace('Unknown', '0')
												.str.extract(r'(\d+|\d+\-)(?!.*\d+)') # extract the last number
												.astype('int')))
											.div(2) 
											.astype('int')
											)



						)                     
			)		

	# default substrings if not provided 
	if substrings is None:
		substrings = [
					'job_role', 
					'other_lang', 
					'other_purposes', 
					'how_involved',
					'web_frameworks', 
					'data_frameworks', 
					'other_frameworks', 
					'database',
					'bigdata', 
					'ide_editor', 
					'configuration_management', 
					'ci',
					'cloud_platform',
					'isolate_environments',
					'unittesting',
					'orm',
					'develop_for_cloud',
					'run_in_cloud',
					'python_version_upgrade',
					'devenv_os'
    				]
		
	# function to combine categorical cols based on substrings
	def combine_cols(data, substrings):
		"""transform categorical-like columns based on the given substrings"""

		transformed_cols = {}

		for substring in substrings:
			transformed_data = (data	
									.filter(axis=1, like=substring)
									.where(data.isna(), '1')
									.fillna('1')
									.astype('int')
									.astype('bool')
									.idxmax(axis=1)
								#	.str.replace(substring, '') # rename the columns 
								)
			
			transformed_cols[substring] = transformed_data

		return transformed_cols
	
	# generate transformed_cols
	transformed_cols = combine_cols(data, substrings)

	# apply transformed columns to the data
	data = data.assign(**transformed_cols)

	# select the relevant columns for the final output
	data = data.loc[:, ['age', 'are_you_datascientist', 'company_size',
						'country_live', 'employment_status',
						'first_learn_about_main_ide', 'how_often_use_main_ide',
						'ide_main', 'is_python_main', 'job_team', 'main_purposes',
						'missing_features_main_ide', 'nps_main_ide', 'python_years',
						'python3_version_most', 'several_projects', 'team_size',
						'use_python_most', 'years_of_coding',
						*transformed_cols.keys()]
				 	]
	
	return data

url = 'https://github.com/mattharrison/datasets/raw/master/data/2020-jetbrains-python-survey.csv'
survey = pd.read_csv(url)

#survey.shape
df = tweak_survey(survey, substrings=None)

# df['company_size'].value_counts(dropna=False)
# df['employment_status'].value_counts(dropna=False)
# df['is_python_main'].value_counts(dropna=False)
# df['team_size'].value_counts(dropna=False)

df.age.value_counts()
# df.sample(20)

C:\Users\fengz\AppData\Local\Temp\ipykernel_16848\2836129855.py:159: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  survey = pd.read_csv(url)


age
0     29701
25     9710
34     7512
44     3010
19     2567
54     1374
60      588
Name: count, dtype: int64

In [152]:
(df
	.groupby(['age', 'country_live'])
    .agg({'python3_version_most': 'size',
          'use_python_most': 'size'
		  })
    .loc[(slice(None), 'United States'), :] #have 2 cols
   
	
	)


(df
	.groupby(['age', 'country_live', 'python3_version_most','use_python_most']) # all indices
    .size()
    .unstack([-1, 2]) #unstack([a,b]) b -> level=0 in cols
    # .loc[(slice(None), ['United States', 'China'], slice(None), slice(None))]
    .loc[(slice(None), ['United States', 'China']), :] #have 2 cols
    .loc[(slice(None), 'United States'), :] #have 2 cols
)


(df
	.groupby(['age', 'country_live', 'python3_version_most','use_python_most']) # all indices
    .size()
    .unstack([-1, 2]) #unstack([a,b]) b -> level=0 in cols
    # .loc[:, (['Data analysis'])]
    .loc[(slice(None), ['United States', 'China']), (['Data analysis', 'Web development'])]
)


(df
	.groupby(['age', 'country_live', 'python3_version_most','use_python_most']) # all indices
    .size()
    .loc[(slice(None), ['United States', 'China'], '3.8', slice(None))]
)

(df
	.groupby(['age', 'country_live', 'python3_version_most','use_python_most']) # all indices
    .size()
    .unstack([-1, 2]) #unstack([a,b]) b -> level=0 in cols
    # .loc[:, (['Data analysis'])]
    .loc[(slice(None), ['United States', 'China']), (['Data analysis', 'Web development'])]
    .swaplevel(axis=0)
	.style.format('{:.0f}').background_gradient(cmap='viridis', axis=None)
)

In [ ]:
()